### Inspekcja modeli PyTorch

In [1]:
# Importy standardowej biblioteki
import sys
import os

# Dodaj katalog główny projektu do sys.path
current_dir = (
    os.path.dirname(os.path.abspath(__file__))
    if "__file__" in globals()
    else os.getcwd()
)
project_root = os.path.abspath(os.path.join(current_dir, "..", ".."))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Katalog główny projektu: {project_root}")
print(f"Czy katalog src istnieje: {os.path.exists(os.path.join(project_root, 'src'))}")

import torch
import torch.serialization

# Importy bibliotek zewnętrznych
import numpy as np

Katalog główny projektu: c:\Users\kubas\Desktop\Projekt dyplomowy\Audio-Emotion-Recognition
Czy katalog src istnieje: True


In [2]:
def add_safe_globals_for_pytorch():
    """
    Dodaje niezbędne klasy/funkcje do bezpiecznej listy PyTorch
    """
    try:
        # Lista potencjalnie wymaganych klas/funkcji
        safe_globals = [
            torch.torch_version.TorchVersion,
            np.core.multiarray._reconstruct,
            np.ndarray,
            np.dtype,
        ]

        # Dodanie wszystkich klas/funkcji do bezpiecznej listy
        for safe_global in safe_globals:
            try:
                torch.serialization.add_safe_globals([safe_global])
            except Exception as e:
                print(f"Nie można dodać do bezpiecznych globali: {e}")

    except Exception as e:
        print(
            f"Ostrzeżenie: Nie można dodać niektórych klas do bezpiecznych globali: {e}"
        )


def safe_inspect_model(path):
    """
    Bezpieczna funkcja do inspekcji zawartości modelu PyTorch

    Args:
        path (str): Ścieżka do pliku .pt z zapisanym modelem
    """
    print(f"Inspekcja pliku modelu: {path}")
    print(f"Wersja PyTorch: {torch.__version__}")

    # Dodanie bezpiecznych globali
    add_safe_globals_for_pytorch()

    # Próba ładowania - najpierw weights_only=False (działa, ale mniej bezpieczne)
    try:
        print("\nPróba ładowania z weights_only=False...")
        state = torch.load(path, map_location=torch.device("cpu"), weights_only=False)
        print("Sukces z weights_only=False!")

        print("\nKlucze dostępne w pliku stanu modelu:")
        for key in state.keys():
            print(f"- {key}")

        print("\nSzczegółowe informacje:")
        for key, value in state.items():
            if isinstance(value, (str, int, float, bool)):
                print(f"{key}: {value}")
            elif isinstance(value, (list, tuple)) and len(value) < 10:
                print(f"{key}: {value}")
            elif isinstance(value, dict) and len(value) < 10:
                print(f"{key}: {value}")
            elif isinstance(value, dict):
                print(f"{key}: {type(value)} z {len(value)} elementami")
                print("  Przykłady:")
                for i, (k, v) in enumerate(value.items()):
                    if i >= 3:
                        break
                    print(f"  - {k}: {type(v)}")
            else:
                print(f"{key}: {type(value)}")

        return state

    except Exception as e:
        print(f"Błąd podczas wczytywania pliku: {str(e)}\n")

        # Jeśli jednak się nie udało, sprawdź metadane JSON
        try:
            metadata_path = path.replace(".pt", "_metadata.json")
            print(f"Próba wczytania metadanych z: {metadata_path}")
            import json

            with open(metadata_path, "r") as f:
                metadata = json.load(f)

            print("\nMetadane modelu:")
            for key, value in metadata.items():
                print(f"- {key}: {value}")

            return metadata
        except Exception as e_meta:
            print(f"Nie udało się wczytać metadanych: {str(e_meta)}")

        return None


model_path = "ensemble_outputs/ensemble_run_20250523_193302/models/ensemble_model.pt"
safe_inspect_model(model_path)

Inspekcja pliku modelu: ensemble_outputs/ensemble_run_20250523_193302/models/ensemble_model.pt
Wersja PyTorch: 2.7.0+cpu

Próba ładowania z weights_only=False...
Sukces z weights_only=False!

Klucze dostępne w pliku stanu modelu:
- model_state_dict
- feature_types
- normalized_weights
- temperature
- regularization_strength
- class_names
- model_version
- pytorch_version

Szczegółowe informacje:
model_state_dict: <class 'collections.OrderedDict'> z 612 elementami
  Przykłady:
  - weights: <class 'torch.Tensor'>
  - temperature: <class 'torch.Tensor'>
  - models.melspectrogram.resnet.conv1.weight: <class 'torch.Tensor'>
feature_types: ['melspectrogram', 'mfcc', 'hpss', 'chroma', 'tempogram']
normalized_weights: {'melspectrogram': 0.14953847229480743, 'mfcc': 0.1552032232284546, 'hpss': 0.35954853892326355, 'chroma': 0.15455292165279388, 'tempogram': 0.1811569184064865}
temperature: 1.0
regularization_strength: 0.01
class_names: ['anger', 'fear', 'happiness', 'neutral', 'sadness', 'surpr

{'model_state_dict': OrderedDict([('weights',
               tensor([0.0335, 0.0706, 0.9108, 0.0664, 0.2253])),
              ('temperature', tensor(1.)),
              ('models.melspectrogram.resnet.conv1.weight',
               tensor([[[[-4.7218e-02,  1.2367e-02, -4.7489e-03,  ...,  5.1557e-02,
                           1.6408e-02,  3.3871e-02],
                         [-3.3177e-02, -3.4345e-02,  3.2446e-03,  ...,  1.6714e-02,
                           2.7851e-02,  3.0250e-02],
                         [ 7.7092e-03, -6.2035e-03, -4.5462e-02,  ..., -2.2799e-02,
                           2.4516e-02,  6.4258e-02],
                         ...,
                         [ 6.4855e-02,  9.7828e-03,  2.9263e-02,  ..., -7.5247e-02,
                           3.9471e-02,  2.5401e-02],
                         [ 5.3840e-02,  2.0648e-02,  6.8153e-02,  ..., -3.4898e-02,
                          -3.3884e-02, -2.1294e-02],
                         [ 1.1208e-02,  3.1149e-02,  3.9437e-02,  ...,